In [6]:
#!pip install rquge
#!pip install transformers
#!pip install sentencepiece
#!pip install git+https://github.com/alirezamshi/RQUGE
#!pip install evaluate
#!pip install sentence-transformers
#!pip install pandas
#!pip install --user sentencepiece
#!pip install git+https://github.com/alirezamshi/RQUGE
#!pip install rouge_score absl-py
#!pip install transformers torch
#!pip install protobuf


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 302 kB 3.1 MB/s eta 0:00:01


In [2]:
import json
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity
from transformers import T5Tokenizer, T5ForConditionalGeneration
from evaluate import load
import pandas as pd
import sentencepiece as spm
from tqdm import tqdm
from datasets import load_metric

In [3]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

In [4]:
#DATASET PREP

In [5]:
#  import sys
# print("Python interpreter:", sys.executable)

# import sentencepiece as sp
# print("SentencePiece location:", sp.__file__)

In [6]:
# with open('/Users/eraparihar/Desktop/Semester 2/SI 630/project/squad_train-v2.0.json', 'r') as file:
#     train_data = json.load(file)

In [7]:
# def convert_to_dataset(data):
#     dataset = []
#     for entry in data['data']:
#         title = entry['title']
#         for paragraph in entry['paragraphs']:
#             context = paragraph['context']
#             for qa in paragraph['qas']:
#                 q = qa['question']
#                 id_ = qa['id']
#                 is_impossible = qa['is_impossible']
#                 answers = qa['answers'] if not is_impossible else []
#                 # For each question, we will create a dictionary with the question, its context and the answers
#                 dataset.append({
#                     'id': id_,
#                     'title': title,
#                     'context': context,
#                     'question': q,
#                     'is_impossible': is_impossible,
#                     'answers': [{'text': answer['text'], 'answer_start': answer['answer_start']} for answer in answers]
#                 })
#     return dataset

In [8]:
# dataset = convert_to_dataset(train_data)

# trial_data = dataset[:100]

In [9]:
with open('trial_data.json', 'r') as file:
    train_data = json.load(file)

In [10]:
#trial_datad = trial_data.to_dict(orient='records') # coverting df to dict
with open('trial_data.json', 'r') as file:
    trial_data = json.load(file)

In [11]:
data = trial_data[0:100]
len(data)

100

In [12]:
# MODEL FOR ANSWER GENERATION

In [13]:

model_name = "allenai/unifiedqa-t5-large" # can specify the model size here
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

def run_model(input_string, **generator_args):
    input_ids = tokenizer.encode(input_string, return_tensors="pt")
    res = model.generate(input_ids, **generator_args)
    return tokenizer.batch_decode(res, skip_special_tokens=True)

# def run_model(input_string, temperature=0.9):
#     input_ids = tokenizer.encode(input_string, return_tensors="pt")
#     res = model.generate(input_ids, **generator_args)
#     return tokenizer.batch_decode(res, skip_special_tokens=True)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [14]:
# Sample
# run_model("which is best conductor? \\n (a) iron (b) feather (c) wood (d) plastic",
#          temperature=0.9, num_return_sequences=4, num_beams=20)

run_model("which is best conductor? \\n (a) iron (b) feather (c) wood (d) plastic",
         temperature=0.9)

/home/erap/.local/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/erap/.local/lib/python3.9/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['iron']

In [15]:
generated_questions = []
contexts = []
generated_answers = []

In [16]:
#following for answer generation evaluation

In [17]:
# for entry in trial_data:
#     input_string = f"Question: {entry['question']} Context: {entry['context']}"
#     output = run_model(input_string)
#     generated_questions.append(output[0])  # Assuming the first item in the output list is the generated question
#     contexts.append(entry['context'])
#     answers.append(entry['answers'][0]['text'])  # Assuming there's at least one answer and we use the first one

for entry in tqdm(data, desc="Processing entries"):
    input_string = f"Question: {entry['question']} Context: {entry['context']}"
    output = run_model(input_string, )
    generated_answers.append(output[0])  
    contexts.append(entry['context'])
    generated_questions.append(entry['question'])

# for df
# for index, entry in tqdm(trial_data.iterrows(), desc="Processing entries", total=trial_data.shape[0]):
#     input_string = f"Question: {entry['question']} Context: {entry['context']}"
#     output = run_model(input_string)
#     generated_questions.append(output[0])  # Assuming the first item in the output list is the generated question
#     contexts.append(entry['context'])
#     # Extracting the text of the first answer assuming the answer format is a list of dictionaries
#     answers.append(entry['answers'][0]['text'])

Processing entries: 100%|██████████| 100/100 [04:48<00:00,  2.88s/it]


In [18]:
generated_answers 

['late 1990s',
 'singing and dancing',
 '2003',
 'Houston, Texas',
 '1990s',
 "Destiny's Child",
 'Dangerously in Love',
 'her father, Mathew Knowles',
 'late 1990s',
 'lead singer',
 'Dangerously in Love',
 '2003',
 'five',
 'lead singer',
 'Dangerously in Love',
 'acting',
 'Jay Z',
 'six',
 'Dreamgirls',
 '2010',
 'Beyoncé',
 'Cadillac Records',
 'June 2005',
 "B'Day",
 'Dreamgirls',
 'Jay Z',
 'Sasha Fierce',
 'love, relationships, and monogamy',
 'influential',
 'Forbes',
 '2000s',
 'Forbes',
 'modern-day feminist',
 '2013 and 2014',
 '118 million',
 '60 million',
 '118 million',
 '20',
 'Forbes',
 "Destiny's Child",
 "her mother's maiden name",
 'African-American',
 'Methodist',
 'Xerox',
 'hairdresser',
 'Solange',
 'Joseph Broussard',
 'Xerox',
 'salon',
 'Solange',
 'Joseph Broussard',
 'Methodist',
 'Fredericksburg',
 'dance instructor Darlette Johnson',
 'Houston',
 'dance instructor Darlette Johnson',
 "St. John's United Methodist Church",
 'music magnet school',
 'Imagine'

In [19]:
generated_questions


['When did Beyonce start becoming popular?',
 'What areas did Beyonce compete in when she was growing up?',
 "When did Beyonce leave Destiny's Child and become a solo singer?",
 'In what city and state did Beyonce  grow up? ',
 'In which decade did Beyonce become famous?',
 'In what R&B group was she the lead singer?',
 'What album made her a worldwide known artist?',
 "Who managed the Destiny's Child group?",
 'When did Beyoncé rise to fame?',
 "What role did Beyoncé have in Destiny's Child?",
 'What was the first album Beyoncé released as a solo artist?',
 'When did Beyoncé release Dangerously in Love?',
 'How many Grammy awards did Beyoncé win for her first solo album?',
 "What was Beyoncé's role in Destiny's Child?",
 "What was the name of Beyoncé's first solo album?",
 'After her second solo album, what other entertainment venture did Beyonce explore?',
 'Which artist did Beyonce marry?',
 'To set the record for Grammys, how many did Beyonce win?',
 'For what movie did Beyonce rec

In [20]:
output

['Independent Women Part I']

In [21]:
input_string


'Question: "Charlie\'s Angels" featured which single from the band members? Context: The remaining band members recorded "Independent Women Part I", which appeared on the soundtrack to the 2000 film, Charlie\'s Angels. It became their best-charting single, topping the U.S. Billboard Hot 100 chart for eleven consecutive weeks. In early 2001, while Destiny\'s Child was completing their third album, Beyoncé landed a major role in the MTV made-for-television film, Carmen: A Hip Hopera, starring alongside American actor Mekhi Phifer. Set in Philadelphia, the film is a modern interpretation of the 19th century opera Carmen by French composer Georges Bizet. When the third album Survivor was released in May 2001, Luckett and Roberson filed a lawsuit claiming that the songs were aimed at them. The album debuted at number one on the U.S. Billboard 200, with first-week sales of 663,000 copies sold. The album spawned other number-one hits, "Bootylicious" and the title track, "Survivor", the latter

In [22]:
contexts

['Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".',
 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead s

In [23]:
data

[{'id': '56be85543aeaaa14008c9063',
  'title': 'Beyoncé',
  'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".',
  'question': 'When did Beyonce start becoming popular?',
  'is_impossible': False,
  'answers': [{'text': 'in the late 1990s', 'answer_start': 269}]},
 {'id': '56be85543aeaaa14008c9065',
  'title': 'Beyoncé',
  'context': 'Beyo

In [24]:
# MODEL FOR QUESTION GENERATION

In [25]:
# trial
tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")
#model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")
model = T5ForConditionalGeneration.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")



def get_question(answer, context, max_length=64):
  input_text = "answer: %s  context: %s </s>" % (answer, context)
  features = tokenizer([input_text], return_tensors='pt')

  output = model.generate(input_ids=features['input_ids'], 
               attention_mask=features['attention_mask'],
               max_length=max_length)

  return tokenizer.decode(output[0])

context = "Manuel has created RuPERTa-base with the support of HF-Transformers and Google"
answer = "Manuel"

get_question(answer, context)    

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


'<pad> question: Who created the RuPERTa-base?</s>'

In [26]:
generated_questions_QG = []
contexts_QG = []
generated_answers_QG = []

In [27]:
model_name = "mrm8488/t5-base-finetuned-question-generation-ap"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


In [28]:
def generate_questions(data):
    generated_questions = []
    for entry in data:
        answer = entry['answers'][0]['text']  
        context = entry['context']
        input_text = f"answer: {answer} context: {context} </s>"
        features = tokenizer(input_text, return_tensors='pt')
        output = model.generate(
            input_ids=features['input_ids'],
            attention_mask=features['attention_mask'],
            max_length=64,
            num_beams=5,
            early_stopping=True
        )
        question = tokenizer.decode(output[0], skip_special_tokens=True)
        generated_questions.append(question)
    return generated_questions

# Generating questions
generated_questions_QG = generate_questions(data)


/home/erap/.local/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5.py:309: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


In [29]:
# for i, question in enumerate(generated_questions, 1):
#     print(f"Generated Question {i}: {question}")
generated_questions_QG

["question: When did Beyonce become famous as the lead singer of Destiny's Child?",
 'question: What competitions did Beyonce perform as a child?',
 'question: When did Beyonce release Dangerously in Love?',
 'question: Where was Beyonce raised?',
 "question: When did Beyonce become famous as the lead singer of Destiny's Child?",
 "question: What was the name of Beyonce's R&B group?",
 "question: What was Beyonce's debut album called?",
 "question: Who managed Destiny's Child?",
 "question: When did Beyonce become famous as the lead singer of Destiny's Child?",
 "question: Beyonce rose to fame as what in Destiny's Child?",
 "question: What was Beyonce's debut album called?",
 'question: When did Beyonce release Dangerously in Love?',
 'question: How many Grammy Awards did Beyonce win for her debut album?',
 "question: Beyonce rose to fame as what in Destiny's Child?",
 "question: What was Beyonce's debut album called?",
 'question: Dreamgirls was a Golden Globe-nominated performance of

In [31]:
# preprocess generated_questions_qg
generated_questions_QG = [q.replace("question: ", "") for q in generated_questions]

# Print the cleaned questions
for question in generated_questions_QG:
    print(question)

When did Beyonce start becoming popular?
What areas did Beyonce compete in when she was growing up?
When did Beyonce leave Destiny's Child and become a solo singer?
In what city and state did Beyonce  grow up? 
In which decade did Beyonce become famous?
In what R&B group was she the lead singer?
What album made her a worldwide known artist?
Who managed the Destiny's Child group?
When did Beyoncé rise to fame?
What role did Beyoncé have in Destiny's Child?
What was the first album Beyoncé released as a solo artist?
When did Beyoncé release Dangerously in Love?
How many Grammy awards did Beyoncé win for her first solo album?
What was Beyoncé's role in Destiny's Child?
What was the name of Beyoncé's first solo album?
After her second solo album, what other entertainment venture did Beyonce explore?
Which artist did Beyonce marry?
To set the record for Grammys, how many did Beyonce win?
For what movie did Beyonce receive  her first Golden Globe nomination?
When did Beyonce take a hiatus in

In [32]:
for entry in tqdm(data):
    contexts_QG.append(entry['context'])
    generated_answers_QG.append(entry['answers'][0]['text'])

100%|██████████| 100/100 [00:00<00:00, 269383.69it/s]


In [33]:
len(generated_answers_QG)

100

In [34]:
# CALCULATE RQUGE SCORE

In [35]:
# for answer generation
rqugescore = load("alirezamsh/rquge")
results = rqugescore.compute(generated_questions=generated_questions, contexts=contexts, answers=generated_answers)
print("Mean RQUGE Score:", results["mean_score"])

Mean RQUGE Score: 4.91074542760849


In [36]:
# for question generation
rqugescore = load("alirezamsh/rquge")
results = rqugescore.compute(generated_questions=generated_questions_QG, contexts=contexts_QG, answers= generated_answers_QG)
print("Mean RQUGE Score:", results["mean_score"])

Mean RQUGE Score: 4.958300240039826


In [23]:
# trial implementation for understanding

# from evaluate import load
# rqugescore = load("alirezamsh/rquge")
# generated_questions = ["how is the weather?"]
# contexts = ["the weather is sunny"]
# answers = ["sunny"]
# results = rqugescore.compute(generated_questions=generated_questions, contexts=contexts, answers=answers)
# print(results["mean_score"])

In [24]:
ground_truth_answers = [item['answers'][0]['text'] for item in data] 



In [25]:
# for ANSWER generation

# Evaluation metrics
bleu_metric = load_metric("bleu")
meteor_metric = load_metric("meteor")
rouge_metric = load_metric("rouge") 

# Calculate BLEU score
bleu_score = bleu_metric.compute(predictions=[generated_question], references=[[ground_truth_answers]])

# Calculate METEOR score
meteor_score = meteor_metric.compute(predictions=generated_answers, references=ground_truth_answers)

# Calculate ROUGE score
rouge_score = rouge_metric.compute(predictions=generated_answers, references=ground_truth_answers)

print("BLEU Score:", bleu_score)
print("METEOR Score:", meteor_score)
print("ROUGE Score:", rouge_score) 

/tmp/ipykernel_1986977/631788243.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bleu_metric = load_metric("bleu")
/home/erap/.local/lib/python3.9/site-packages/datasets/load.py:756: FutureWarning: The repository for bleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/bleu/bleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/home/erap/.local/lib/python3.9/site-packages/datasets/load.py:756: FutureWarning: The repository for meteor contains custom code which must be executed to correctly load the metric. You can inspect the

BLEU Score: {'bleu': 1.0, 'precisions': [1.0, 1.0, 1.0, 1.0], 'brevity_penalty': 1.0, 'length_ratio': 1.0, 'translation_length': 100, 'reference_length': 100}
METEOR Score: {'meteor': 0.7966542361111112}
ROUGE Score: {'rouge1': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)), 'rouge2': AggregateScore(low=Score(precision=0.56, recall=0.56, fmeasure=0.56), mid=Score(precision=0.66, recall=0.66, fmeasure=0.66), high=Score(precision=0.75, recall=0.75, fmeasure=0.75)), 'rougeL': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)), 'rougeLsum': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0))}


In [1]:
ground_truth_question = [item['question'] for item in data] 


NameError: name 'data' is not defined

In [ ]:
# for QUESTION generation

# Evaluation metrics
bleu_metric = load_metric("bleu")
meteor_metric = load_metric("meteor")
rouge_metric = load_metric("rouge") 

# Calculate BLEU score
bleu_score = bleu_metric.compute(predictions=[generated_questions_QG], references=[[ground_truth_answers]])

# Calculate METEOR score
meteor_score = meteor_metric.compute(predictions=generated_questions_QG, references=ground_truth_answers)

# Calculate ROUGE score
rouge_score = rouge_metric.compute(predictions=generated_questions_QG, references=ground_truth_answers)

print("BLEU Score:", bleu_score)
print("METEOR Score:", meteor_score)
print("ROUGE Score:", rouge_score) 

In [26]:
# trial

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("potsawee/t5-large-generation-race-QuestionAnswer")
model = AutoModelForSeq2SeqLM.from_pretrained("potsawee/t5-large-generation-race-QuestionAnswer")

context = r"""
World number one Novak Djokovic says he is hoping for a "positive decision" to allow him 
to play at Indian Wells and the Miami Open next month. The United States has extended 
its requirement for international visitors to be vaccinated against Covid-19. Proof of vaccination 
will be required to enter the country until at least 10 April, but the Serbian has previously 
said he is unvaccinated. The 35-year-old has applied for special permission to enter the country. 
Indian Wells and the Miami Open - two of the most prestigious tournaments on the tennis calendar 
outside the Grand Slams - start on 6 and 20 March respectively. Djokovic says he will return to 
the ATP tour in Dubai next week after claiming a record-extending 10th Australian Open title 
and a record-equalling 22nd Grand Slam men's title last month.""".replace("\n", "")

inputs = tokenizer(context, return_tensors="pt")
outputs = model.generate(**inputs, max_length=100)
question_answer = tokenizer.decode(outputs[0], skip_special_tokens=False)
question_answer = question_answer.replace(tokenizer.pad_token, "").replace(tokenizer.eos_token, "")
question, answer = question_answer.split(tokenizer.sep_token)

print("question:", question)
print("answer:", answer)


question:  What is the best title for the passage?
answer:  Djokovic's application for special permission to enter the United States
